In [19]:
# Ziwei Qiu, ziweiqiu@g.harvard.edu
from dimod import BinaryQuadraticModel
from dimod import ExactSolver
from neal import SimulatedAnnealingSampler
from itertools import combinations
# from dwave.system import LeapHybridBQMSampler
import numpy as np

## Problem Definition

The set cover problem is to identify the smallest sub-collection of `V` whose union equals the universe `U`.

In [3]:
# Define a simple set cover problem
U = list(set(np.random.randint(10, size=(5))))
# U = list(set([1,2,4]))


V = [set(np.random.randint(10, size=(5))) for i in range(5)]
# V = [set([1]),set([1,2]),set([4]), set([2])]

print('The universe is',U)
print('There are {:d} collections:'.format(len(V)),V)

print('Number of elements in the universe: {:d}'.format(len(U)))
print('Number of sets: N={:d}'.format(len(V)))

# Create indicator variables
I = []
for i in range(len(V)):
    I.append([1 if U[a] in V[i] else 0 for a in range(len(U))])
print('Indicator variables: I_i,a',I)

# Lagrange multipliers A>B>0
A = 2
B = 1

The universe is [4, 6, 7]
There are 5 collections: [{0, 3, 4}, {8, 2, 5, 7}, {8, 3, 4, 7}, {9, 4, 5, 6}, {8, 9, 2, 4}]
Number of elements in the universe: 3
Number of sets: N=5
Indicator variables: I_i,a [[1, 0, 0], [0, 0, 1], [1, 0, 1], [1, 1, 0], [1, 0, 0]]


## Create the BQM model

In [6]:
##@  Binary Quadratic Model @##
bqm = BinaryQuadraticModel('BINARY')

# Add linear terms
# x linear terms
x = [bqm.add_variable(f'x_{i+1}', A*sum(I[i])+B) for i in range(0,len(V))]
print('x variables:',x)

# y_am linear terms
y = []
for a in range(1,len(U)+1):
    y.append([bqm.add_variable(f'y_{a,m}', A*(m**2-1)) for m in range(1,len(V)+1)])
print('y variables:',y)

# Add quadratic terms

# x_i-x_j terms
for i in range(1,len(V)+1):
    for j in range(i+1,len(V)+1):
        key = ('x_' + str(i), 'x_' + str(j))
        bqm.quadratic[key] = 2*A*np.dot(np.array(I[i-1]),np.array(I[j-1]))
        
# y_am - y_an terms
for m in range(1,len(V)+1):
    for n in range(m+1,len(V)+1):
        for a in range(1,len(U)+1):
            key = ('y_('+str(a)+', '+str(m)+')', 'y_('+str(a)+', '+str(n)+')')
            bqm.quadratic[key] = 2*A*(1+m*n)
            
# x_i-y_am terms
for i in range(1,len(V)+1):
    for m in range(1,len(V)+1):
        for a in range(1,len(U)+1):
            key = ('x_' + str(i), 'y_('+str(a)+', '+str(m)+')')
            bqm.quadratic[key] = -2*A*m*I[i-1][a-1]

x variables: ['x_1', 'x_2', 'x_3', 'x_4', 'x_5']
y variables: [['y_(1, 1)', 'y_(1, 2)', 'y_(1, 3)', 'y_(1, 4)', 'y_(1, 5)'], ['y_(2, 1)', 'y_(2, 2)', 'y_(2, 3)', 'y_(2, 4)', 'y_(2, 5)'], ['y_(3, 1)', 'y_(3, 2)', 'y_(3, 3)', 'y_(3, 4)', 'y_(3, 5)']]


## Exact Solver

In [ ]:
response = ExactSolver().sample(bqm)
samples = response.samples()
best_solution = response.first.sample
print([best_solution[i] for i in x])

## SimulatedAnnealingSampler

In [14]:
response = SimulatedAnnealingSampler().sample(bqm, num_reads=100, num_sweeps=10000).truncate(10)
# response = SimulatedAnnealingSampler().sample(bqm).truncate(10)
print(response)
best_solution = response.first.sample
print([best_solution[i] for i in x])
# print(best_solution)

  x_1 x_2 x_3 x_4 x_5 y_(1, 1) y_(1, 2) y_(1, 3) ... y_(3, 5) energy num_oc.
0   0   0   1   1   0        0        1        0 ...        0   -4.0       1
1   0   0   1   1   0        0        1        0 ...        0   -4.0       1
2   0   1   0   1   0        1        0        0 ...        0   -4.0       1
3   0   0   1   1   0        0        1        0 ...        0   -4.0       1
4   0   0   1   1   0        0        1        0 ...        0   -4.0       1
5   0   0   1   1   0        0        1        0 ...        0   -4.0       1
6   0   1   0   1   0        1        0        0 ...        0   -4.0       1
7   0   1   0   1   0        1        0        0 ...        0   -4.0       1
8   0   1   0   1   0        1        0        0 ...        0   -4.0       1
9   0   1   0   1   0        1        0        0 ...        0   -4.0       1
['BINARY', 10 rows, 10 samples, 20 variables]
[0, 0, 1, 1, 0]


## LeapHybridDQMSampler

In [17]:
response = LeapHybridBQMSampler().sample_bqm(bqm, num_reads=100, num_sweeps=10000).truncate(10)
# response = SimulatedAnnealingSampler().sample(bqm).truncate(10)
print(response)
best_solution = response.first.sample
print([best_solution[i] for i in x])

AttributeError: 'LeapHybridDQMSampler' object has no attribute 'sample_bqm'